# LeetCode 181: Employees Earning More Than Their Managers

### Problem Statement

**Table: Employee**

| Column Name | Type    |
|-------------|---------|
| id          | int     |
| name        | varchar |
| salary      | int     |
| managerId   | int     |

`id` is the primary key (column with unique values) for this table.
Each row of this table indicates the ID of an employee, their name, salary, and the ID of their manager.

**Task:**
Write a solution to find the employees who earn more than their managers.

**Example 1:**

**Input:**
Employee table:
| id | name  | salary | managerId |
|----|-------|--------|-----------|
| 1  | Joe   | 70000  | 3         |
| 2  | Henry | 80000  | 4         |
| 3  | Sam   | 60000  | Null      |
| 4  | Max   | 90000  | Null      |

**Output:**
| Employee |
|----------|
| Joe      |

**Explanation:** Joe is the only employee who earns more than his manager.

In [4]:
import sqlite3
import pandas as pd

# 1. Setup SQLite in-memory database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 2. Helper function to display query results as a Pandas DataFrame
def show(query):
    return pd.read_sql_query(query, conn)

print("Environment setup complete. Database ready.")

Environment setup complete. Database ready.


### Schema Description

The schema consists of a single table, `Employee`.

This is a classic **Unary Relationship** (or Recursive Relationship). In database theory, this occurs when an entity type has a relationship with itself.

*   **Primary Key (`id`):** Uniquely identifies an employee.
*   **Foreign Key (`managerId`):** References the `id` column *in the same table*.
*   **Cardinality:** One-to-Many (1:N). One manager can manage many employees, but an employee typically reports to only one manager (in this simplified schema).

In [5]:
# Create the Employee table
create_table_sql = """
CREATE TABLE Employee (
    id INTEGER PRIMARY KEY,
    name VARCHAR(255),
    salary INTEGER,
    managerId INTEGER
);
"""

cursor.execute(create_table_sql)
conn.commit()
print("Table 'Employee' created successfully.")

Table 'Employee' created successfully.


### Sample Data

We will populate the database with the example data provided in the LeetCode problem.

**Data Hierarchy Analysis:**
1.  **Joe (id:1)** -> Reports to **Sam (id:3)**
2.  **Henry (id:2)** -> Reports to **Max (id:4)**
3.  **Sam (id:3)** -> Reports to **NULL** (Top of hierarchy)
4.  **Max (id:4)** -> Reports to **NULL** (Top of hierarchy)

In [6]:
# Insert sample data
insert_data_sql = """
INSERT INTO Employee (id, name, salary, managerId) VALUES
(1, 'Joe', 70000, 3),
(2, 'Henry', 80000, 4),
(3, 'Sam', 60000, NULL),
(4, 'Max', 90000, NULL);
"""

cursor.execute(insert_data_sql)
conn.commit()
print("Sample data inserted.")

Sample data inserted.


### 🎓 Lecture: Self-Joins, Recursive Relationships, and ER Modeling

As a Senior Data Professional, you must master the concept of the **Self-Join**. While simple in syntax, it represents a fundamental concept in Graph Theory and Hierarchical Data modeling within relational databases.

#### 1. The Data Model: Why Single-Table?
Junior engineers often propose splitting this data into two tables: `Employees` and `Managers`.

**Why is this a bad design?**
*   **Normalization Violation:** A Manager is also a human with a `Name` and `Salary`. If we split tables, we duplicate attributes.
*   **Data Integrity:** If "Joe" gets promoted, you would have to DELETE from `Employees` and INSERT into `Managers`. This is risky and complex transactionally.
*   **The Solution:** Single Table Inheritance. The `managerId` acts as a pointer to the *same* table's Primary Key.

**ASCII ER Diagram (Entity-Relationship):**
This diagram shows the "Unary" relationship.

       +-------------------------+
       |        Employee         |
       +-------------------------+
       | PK  id                  | <-------+
       |     name                |         |
       |     salary              |         |
       | FK  managerId           | --------+
       +-------------------------+    |
                |                     |
                | (Child)             | (Parent)
                | Reports To          | Manages
                |                     |
              +---+                 /---\
              |   | (0,1)           |   | (0,N)
              +---+                 \---/

*   **(0,1):** An employee reports to 0 or 1 Manager.
*   **(0,N):** A Manager manages 0 to N Employees.

#### 2. The Mechanics of the Self-Join
SQL does not understand "hierarchy" natively in a standard `SELECT`. To compare a child node to a parent node, we must flatten the relationship.

We use **Aliasing** to trick the SQL engine into thinking it has two copies of the table:
1.  **Table `Emp`:** The perspective of the subordinate.
2.  **Table `Mgr`:** The perspective of the boss.

**ASCII Join Visualization:**
We slide the `Mgr` table until `Mgr.id` aligns with `Emp.managerId`.

    STEP 1: The Raw Data (Vertical)
    -------------------------------
    id: 1 | Name: Joe   | Salary: 70k | Mgr: 3
    id: 3 | Name: Sam   | Salary: 60k | Mgr: NULL

    STEP 2: The Self-Join (Horizontal Alignment)
            Query: FROM Employee Emp JOIN Employee Mgr ON Emp.managerId = Mgr.id

          [ Emp View (Subordinate) ]                 [ Mgr View (Boss) ]
    +----+-------+--------+-----------+        +----+-------+--------+-----------+
    | id | name  | salary | managerId |  --->  | id | name  | salary | managerId |
    +----+-------+--------+-----------+        +----+-------+--------+-----------+
    | 1  | Joe   | 70k    | 3         | MATCH  | 3  | Sam   | 60k    | NULL      |
    +----+-------+--------+-----------+        +----+-------+--------+-----------+
                                         ^
                                         |
                           This horizontal row now contains
                           BOTH salaries for comparison.

#### 3. Relational Algebra & Null Handling
Why use `JOIN` (Inner) instead of `LEFT JOIN`?

*   **The Logic:** If an employee has `managerId = NULL` (like the CEO), they have no one to earn more than.
*   **Inner Join Behavior:** `NULL = 3` evaluates to `UNKNOWN` (effectively False). The row is dropped.
*   **Left Join Behavior:** The row is kept, but the Manager columns become `NULL`.
    *   Comparison: `70000 > NULL`.
    *   Result: `UNKNOWN`.
    *   While correct (it won't return true), `INNER JOIN` is more performant as it filters the dataset earlier in the query plan.

#### 4. Performance in Production
In a table with 10 Million rows (e.g., Walmart's global workforce), this query can be slow without proper indexing.

*   **Naive Approach:** The database scans the entire table to find "Manager ID 3" for "Employee 1". Complexity: $O(N^2)$.
*   **Optimized Approach:** Create an index on `managerId`.
    
        CREATE INDEX idx_manager ON Employee(managerId);
    
    This turns the lookup into a B-Tree search, reducing complexity to $O(N \log N)$ or $O(N)$ depending on the hash join strategy used by the optimizer.

### Step-by-Step Reasoning for the Solution

We need to output the names of employees who have a higher salary than their designated manager.

**Logical Steps:**

1.  **Select Source:** We need data from the `Employee` table.
2.  **Comparison Requirement:** We need to compare `Salary(Employee)` vs `Salary(Manager)`.
3.  **Mechanism:** Since both values reside in the same table, we perform a Self-Join.
    *   Let `E` represent the Employee role.
    *   Let `M` represent the Manager role.
4.  **Join Condition:** Connect the employee to their manager.
    *   `E.managerId` (The employee's pointer to the boss) MUST EQUAL `M.id` (The boss's primary identity).
5.  **Filter Condition:** `E.salary > M.salary`.
6.  **Projection:** Select only `E.name` and rename the column header to `Employee` as requested.

**Drafting the Query:**

    SELECT
        E.name AS Employee
    FROM
        Employee AS E
    JOIN
        Employee AS M
    ON
        E.managerId = M.id
    WHERE
        E.salary > M.salary;

In [7]:
# Final SQL Solution
final_query = """
SELECT
    E.name AS Employee
FROM
    Employee AS E
JOIN
    Employee AS M ON E.managerId = M.id
WHERE
    E.salary > M.salary;
"""

### Output Verification

We expect the query to compare:
1.  **Joe (70k)** vs **Sam (60k)** -> Joe > Sam -> **Keep Joe**
2.  **Henry (80k)** vs **Max (90k)** -> Henry < Max -> Drop Henry
3.  **Sam (60k)** vs **NULL** -> No Manager -> Drop Sam (Inner Join)
4.  **Max (90k)** vs **NULL** -> No Manager -> Drop Max (Inner Join)

**Expected Output:**
| Employee |
|----------|
| Joe      |

In [8]:
# Execute and show final results
result = show(final_query)
result

,Employee
0,Joe


### Summary and Key Takeaways

1.  **Self-Joins allow vertical data to be compared horizontally.** By joining a table to itself, we aligned the `Employee` row next to the `Manager` row.
2.  **Aliasing is mandatory.** In a self-join, you must provide unique aliases (`E` and `M`) so the SQL engine knows which instance of the table you are referring to.
3.  **Handling Top-Level Nodes.** The `INNER JOIN` implicitly filtered out the top-level managers (who have `NULL` managerIds) because `NULL` cannot equal an `id`. This was desirable behavior for this specific logic.
4.  **Schema Design.** This problem validates the "Adjacency List" model for storing hierarchies in relational databases, where a parent reference is stored in the child row.